In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### import libs


In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow.keras as k

import csv
import cv2
import os

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications import VGG16,VGG19
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing import image

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.model_selection import train_test_split

from scipy import ndimage

In [22]:
imagetype= os.listdir("../input/emotion-detection-fer/train")
print(imagetype)

In [23]:
images = []    #images list will store all images
path = "../input/emotion-detection-fer/train"
labels= []

In [24]:
for i in imagetype:
    print(i)
    flower_path=path+"/"+str(i)
    data_list=[j for j in os.listdir(flower_path) if j.endswith(".png") ]
    counter=0
    for data in data_list:
        if counter> 435: break
        image=cv2.imread(flower_path + '/'+ data)
        image=cv2.resize(image,(48,48)) #resize the imag
        img = cv2.Canny(image,100,200)
        images.append(img)
        labels.append(i)
        counter = counter+1
    
print("total images:",len(images))

In [25]:
encode= LabelEncoder()
labels=encode.fit_transform(labels)
labels

In [26]:
labels=labels.reshape(-1,1)
onehotencoder= OneHotEncoder()
labels=onehotencoder.fit_transform(labels)
labels=labels.toarray()
labels

In [27]:
images=np.array(images)
print(images.shape)
print(labels.shape)

In [28]:
images,labels = shuffle(images, labels, random_state=1)


In [29]:
x_train , x_test , y_train , y_test = train_test_split(images, labels, test_size=0.1 , random_state=415)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [30]:
plt.imshow(images[6])
plt.xlabel(labels[6])

In [31]:
DenseNet201 = DenseNet201(input_shape= (48,48,3) ,  include_top=False , weights="imagenet", pooling='max')


In [32]:
DenseNet201.trainable = False

In [33]:
x = Flatten()(DenseNet201.output)


In [34]:
prediction = Dense(7, activation='softmax')(x)

# create a model object
model = Model(inputs=DenseNet201.input, outputs=prediction)

In [35]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [36]:
r = model.fit(x_train , y_train , batch_size=32 , epochs=5 , validation_data = (x_test , y_test))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
r = model.fit(x_train , y_train , batch_size=32 , epochs=5 , validation_data = (x_test , y_test))

In [ ]:
r = model.fit(x_train , y_train , batch_size=64 , epochs=10 , validation_data = (x_test , y_test))

In [ ]:
mobile = MobileNet(input_shape= (48,48,3) , dropout = 0.01 ,  include_top=False , weights="places205", pooling='max')

In [ ]:
mobile.trainable = False

In [ ]:
x = Flatten()(mobile.output)

In [ ]:
prediction = Dense(7, activation='softmax')(x)

# create a model object
mobile = Model(inputs=mobile.input, outputs=prediction)

In [ ]:
mobile.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
r = model.fit(x_train , y_train , batch_size=8 , epochs=10 , validation_data = (x_test , y_test))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
r = model.fit(x_train , y_train , batch_size=64 , epochs=10 , validation_data = (x_test , y_test))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

### canny edge detection
1. Noise reduction;
2. Gradient calculation;
3. Non-maximum suppression;
4. Double threshold;
5. Edge Tracking by Hysteresis.

In [20]:
img = cv2.imread('../input/emotion-detection-fer/train/angry/im1016.png',0)
edges = cv2.Canny(img,100,200)
plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
plt.show()

#### Noise reduction:

In [56]:
def gaussian_kernel(size, sigma=1):
    size = int(size) // 2
    x, y = np.mgrid[-size:size+1, -size:size+1]
    normal = 1 / (2.0 * np.pi * sigma**2)
    g =  np.exp(-((x**2 + y**2) / (2.0*sigma**2))) * normal
    return g

In [60]:
image = gaussian_kernel(1)
# im= image * images[0]
imagess=plt.imread("../input/emotion-detection-fer/train/angry/im0.png")
# im= image * imagess


In [59]:
plt.imshow(imagess)
plt.imshow(im)


#### Gradient Calculation


In [9]:
from scipy import ndimage

def sobel_filters(img):
    Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)
    
    Ix = ndimage.filters.convolve(img, Kx)
    Iy = ndimage.filters.convolve(img, Ky)
    
    G = np.hypot(Ix, Iy)
    G = G / G.max() * 255
    theta = np.arctan2(Iy, Ix)
    
    return (G, theta)

In [ ]:
sobel = sobel

#### Non-Maximum Suppression


In [10]:
def non_max_suppression(img, D):
    M, N = img.shape
    Z = np.zeros((M,N), dtype=np.int32)
    angle = D * 180. / np.pi
    angle[angle < 0] += 180

    
    for i in range(1,M-1):
        for j in range(1,N-1):
            try:
                q = 255
                r = 255
                
               #angle 0
                if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                    q = img[i, j+1]
                    r = img[i, j-1]
                #angle 45
                elif (22.5 <= angle[i,j] < 67.5):
                    q = img[i+1, j-1]
                    r = img[i-1, j+1]
                #angle 90
                elif (67.5 <= angle[i,j] < 112.5):
                    q = img[i+1, j]
                    r = img[i-1, j]
                #angle 135
                elif (112.5 <= angle[i,j] < 157.5):
                    q = img[i-1, j-1]
                    r = img[i+1, j+1]

                if (img[i,j] >= q) and (img[i,j] >= r):
                    Z[i,j] = img[i,j]
                else:
                    Z[i,j] = 0

            except IndexError as e:
                pass
    
    return Z


#### Double threshold


In [11]:
def threshold(img, lowThresholdRatio=0.05, highThresholdRatio=0.09):
    
    highThreshold = img.max() * highThresholdRatio;
    lowThreshold = highThreshold * lowThresholdRatio;
    
    M, N = img.shape
    res = np.zeros((M,N), dtype=np.int32)
    
    weak = np.int32(25)
    strong = np.int32(255)
    
    strong_i, strong_j = np.where(img >= highThreshold)
    zeros_i, zeros_j = np.where(img < lowThreshold)
    
    weak_i, weak_j = np.where((img <= highThreshold) & (img >= lowThreshold))
    
    res[strong_i, strong_j] = strong
    res[weak_i, weak_j] = weak
    
    return (res, weak, strong)

#### Edge Tracking by Hysteresis


In [12]:
def hysteresis(img, weak, strong=255):
    M, N = img.shape  
    for i in range(1, M-1):
        for j in range(1, N-1):
            if (img[i,j] == weak):
                try:
                    if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                        or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                        or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                        img[i, j] = strong
                    else:
                        img[i, j] = 0
                except IndexError as e:
                    pass
    return img